In [ ]:
!pip install tensorflow

In [ ]:
# Step 1: Import Libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, callbacks, applications
import matplotlib.pyplot as plt
import numpy as np

# Step 2: Load and Preprocess CIFAR-10 Dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to range [0, 1]
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define class names for visualization
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Step 3: 建立模型 - 遷移學習
# 載入預訓練的 MobileNetV2 模型，不包含頂部的分類層
# weights='imagenet' -> 使用在 ImageNet 上訓練好的權重
# input_shape -> 我們影像的尺寸
# include_top=False -> 我們要自己定義分類層
base_model = applications.MobileNetV2(
    input_shape=(96, 96, 3), # Corrected input shape to match the upsampled image
    include_top=False,
    weights='imagenet'
)

# 凍結預訓練模型的權重，在第一階段訓練時我們不去動它
base_model.trainable = False

# 建立我們自己的模型
# 我們需要將 32x32 的影像放大，因為 MobileNetV2 在較大影像上表現更好
# 這裡我們將其放大到 96x96
inputs = layers.Input(shape=(32, 32, 3))
x = layers.UpSampling2D(size=(3,3))(inputs) # 放大影像
x = base_model(x, training=False) # 確保 base_model 在推論模式
x = layers.GlobalAveragePooling2D()(x) # 將特徵圖轉換為向量
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x) # 加入 Dropout 防止過擬合
outputs = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs, outputs)

# 顯示模型摘要 (只會顯示我們自己加的層和總參數)
# model.summary() # This is commented out as the original did not print it twice before the final summary.


# Step 4: 編譯模型 (第一階段)
# 使用較高的學習率來訓練我們新加的分類層
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: 定義回呼函式 (Callbacks)
# 除了 EarlyStopping，我們再加入學習率動態調整
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6, # 耐心設短一點，因為我們有兩個訓練階段
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, # 當學習停滯時，將學習率乘以 0.2
    patience=2,
    min_lr=0.00001 # 學習率最小不低於此值
)

# Step 6: 訓練模型 (第一階段 - 只訓練分類頭)
print("--- 訓練分類頭 ---")
history = model.fit(train_images, train_labels,
                    epochs=12, # 第一階段不需要太多 epochs
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping, reduce_lr])


# Step 7: Evaluate and Print Performance Summary
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
training_epochs = len(history.history['accuracy'])

print("\nModel Performance Summary:")
print(f"  Test Accuracy: {test_acc:.4f}")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Final Training Accuracy: {final_train_acc:.4f}")
print(f"  Final Validation Accuracy: {final_val_acc:.4f}")
print(f"  Final Training Loss: {final_train_loss:.4f}")
print(f"  Final Validation Loss: {final_val_loss:.4f}")
print(f"  Training Epochs: {training_epochs}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
--- 訓練分類頭 ---
Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 38s 18ms/step - accuracy: 0.6151 - loss: 1.1363 - val_accuracy: 0.7543 - val_loss: 0.6970 - learning_rate: 0.0010
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.7363 - loss: 0.7702 - val_accuracy: 0.7710 - val_loss: 0.6529 - learning_rate: 0.0010
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.7555 - loss: 0.7104 - val_accuracy: 0.7756 - val_loss: 0.6458 - learning_rate: 0.0010
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.7674 - loss: 0.6758 - val_accuracy: 0.7900 - val_loss: 0.6170 - learning_rate: 0.0010
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.7781 - loss: 0.6373 - val_accuracy: 0.7940 - val_loss: 0.6026 - learning_rate: 0.0010
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.7877 - loss: 0.6079 - val_accuracy: 0